In [2]:
import pandas as pd

Data cleaning jobs:
- other tables and inserting IDs instead of taking up space with strings?
    - users, developers, platform, genres, publishers
- Do tags need to exist in the review table to carry over to the recommendation system?
    - thus does the review table need the genre tags, game title, platform, etc. in order to filter options?
    - or can they be brought in from the other tables?
- null values: blank fields from an issue picking up the data? or from no data to pick up?
- date column as date data type

## Games Table

In [3]:
games = pd.read_csv('games.csv')
games = games.drop_duplicates()
games = games.reset_index(drop=True)
games.shape

(16950, 11)

#### Generate Game ID on game table and join on review table:

In [4]:
# add game_id column
ids = pd.Series(range(10000, 26950))
games = pd.concat((ids, games), axis=1)
games.head()

,0,ESRB_rating,average_user_score,developer,genres,metascore,platform,publisher,release_date,summary,title,url
0,10000,T,7.6,Criterion Games,"Driving, Racing, Arcade",94.0,Xbox,EA Games,"Sep 7, 2004",Burnout 3 challenges you to crash into (and th...,Burnout 3: Takedown,https://www.metacritic.com/game/xbox/burnout-3...
1,10001,T,8.7,Flight-Plan,"Strategy, General",65.0,PlayStation 2,Atlus Co.,"Nov 11, 2008",Our world descended into chaos the day the dem...,Eternal Poison,https://www.metacritic.com/game/playstation-2/...
2,10002,E,4.6,Proper Games,"Miscellaneous, Puzzle, Action, Puzzle, General...",65.0,PC,Capcom,"Apr 7, 2009",FLOCK! uses a sophisticated physics engine tha...,Flock!,https://www.metacritic.com/game/pc/flock!
3,10003,NaN,6.8,Pendulo Studios,"Adventure, General, General, Point-and-Click",65.0,PC,Focus Home Interactive,"Mar 22, 2012","(Also known as ""New York Crimes"") In New York ...",Yesterday,https://www.metacritic.com/game/pc/yesterday
4,10004,NaN,7.6,damiansommer,"Adventure, General, General",66.0,PC,damiansommer,"Jun 14, 2013",The evil Yawhg is returning. How will the town...,The Yawhg,https://www.metacritic.com/game/pc/the-yawhg


In [5]:
# re-name game_ID column and re-arrange columns for easier reading
games.columns = ['game_ID',        'ESRB_rating', 'average_user_score',
                'developer',             'genres',          'metascore',
                 'platform',          'publisher',       'release_date',
                  'summary',              'title',                'url']
games = games[['game_ID', 'title', 'platform', 'metascore', 'average_user_score', 'summary', 'genres',
'developer',  'publisher', 'release_date','ESRB_rating', 'url']]
games.head()

,game_ID,title,platform,metascore,average_user_score,summary,genres,developer,publisher,release_date,ESRB_rating,url
0,10000,Burnout 3: Takedown,Xbox,94.0,7.6,Burnout 3 challenges you to crash into (and th...,"Driving, Racing, Arcade",Criterion Games,EA Games,"Sep 7, 2004",T,https://www.metacritic.com/game/xbox/burnout-3...
1,10001,Eternal Poison,PlayStation 2,65.0,8.7,Our world descended into chaos the day the dem...,"Strategy, General",Flight-Plan,Atlus Co.,"Nov 11, 2008",T,https://www.metacritic.com/game/playstation-2/...
2,10002,Flock!,PC,65.0,4.6,FLOCK! uses a sophisticated physics engine tha...,"Miscellaneous, Puzzle, Action, Puzzle, General...",Proper Games,Capcom,"Apr 7, 2009",E,https://www.metacritic.com/game/pc/flock!
3,10003,Yesterday,PC,65.0,6.8,"(Also known as ""New York Crimes"") In New York ...","Adventure, General, General, Point-and-Click",Pendulo Studios,Focus Home Interactive,"Mar 22, 2012",NaN,https://www.metacritic.com/game/pc/yesterday
4,10004,The Yawhg,PC,66.0,7.6,The evil Yawhg is returning. How will the town...,"Adventure, General, General",damiansommer,damiansommer,"Jun 14, 2013",NaN,https://www.metacritic.com/game/pc/the-yawhg


#### Missingness?

In [6]:
games.isnull().sum()
# Not all games had enough reviews for scores (much more common for user scores)
# Not all games had summaries
# Not all games had ratings

game_ID                  0
title                    0
platform                 0
metascore                7
average_user_score    1219
summary                152
genres                   0
developer                0
publisher                0
release_date             0
ESRB_rating           1547
url                      0
dtype: int64

#### Creating Genres table, Creating table to identify genre tags for each game

In [7]:
# grabbing all genres from games table
genres = []
for row in games.genres:
    genre_list = row.split(sep = ', ')
    for item in genre_list:
        if item not in genres:
            genres.append(item)


# creating genre table
genres = pd.DataFrame(genres)
ids = pd.Series(range(1, 170))
genres = pd.concat((genres, ids), axis=1)
genres.columns = ['genre_name', 'genre_ID']
genres.head()

,genre_name,genre_ID
0,Driving,1
1,Racing,2
2,Arcade,3
3,Strategy,4
4,General,5


In [8]:
# Creating table to identify genre tags for each game (for easy querying)
game_genre_tags = pd.DataFrame(columns = ['game_ID', 'genre_name'])

for index, row in games.iterrows():
    genre_list = row[6].split(sep = ', ')
    for item in genre_list:
        df = pd.DataFrame([[row[0], item]], columns = ['game_ID', 'genre_name'])
        game_genre_tags = game_genre_tags.append(df)
game_genre_tags.head()

,game_ID,genre_name
0,10000,Driving
0,10000,Racing
0,10000,Arcade
0,10001,Strategy
0,10001,General


In [9]:
game_genre_tags = game_genre_tags.drop_duplicates()
game_genre_tags = game_genre_tags.reset_index(drop=True)
game_genre_tags.head()

,game_ID,genre_name
0,10000,Driving
1,10000,Racing
2,10000,Arcade
3,10001,Strategy
4,10001,General


In [10]:
game_genre_tags.shape

(57504, 2)

In [11]:
#Join genres and game genre tags
game_genre_tags = pd.merge(game_genre_tags, genres, how = "left", on = ['genre_name'])

#Drop genre_name column
game_genre_tags = game_genre_tags.drop(['genre_name'], 1)
game_genre_tags.head()

,game_ID,genre_ID
0,10000,1
1,10000,2
2,10000,3
3,10001,4
4,10001,5


In [12]:
game_genre_tags.shape

(57504, 2)

In [13]:
#### Update Game Table
# Drop unnecessary columns from game table
games = games.drop(['genres'], 1)
games.head()

,game_ID,title,platform,metascore,average_user_score,summary,developer,publisher,release_date,ESRB_rating,url
0,10000,Burnout 3: Takedown,Xbox,94.0,7.6,Burnout 3 challenges you to crash into (and th...,Criterion Games,EA Games,"Sep 7, 2004",T,https://www.metacritic.com/game/xbox/burnout-3...
1,10001,Eternal Poison,PlayStation 2,65.0,8.7,Our world descended into chaos the day the dem...,Flight-Plan,Atlus Co.,"Nov 11, 2008",T,https://www.metacritic.com/game/playstation-2/...
2,10002,Flock!,PC,65.0,4.6,FLOCK! uses a sophisticated physics engine tha...,Proper Games,Capcom,"Apr 7, 2009",E,https://www.metacritic.com/game/pc/flock!
3,10003,Yesterday,PC,65.0,6.8,"(Also known as ""New York Crimes"") In New York ...",Pendulo Studios,Focus Home Interactive,"Mar 22, 2012",NaN,https://www.metacritic.com/game/pc/yesterday
4,10004,The Yawhg,PC,66.0,7.6,The evil Yawhg is returning. How will the town...,damiansommer,damiansommer,"Jun 14, 2013",NaN,https://www.metacritic.com/game/pc/the-yawhg


#### Misc tables?

In [14]:
#### Create Platform Table
platforms = pd.DataFrame(games.platform.drop_duplicates())
platforms = platforms.reset_index(drop=True)
ids = pd.Series(range(1, 21))
platforms = pd.concat((platforms, ids), axis=1)
platforms.columns = ['platform', 'platform_ID']
platforms.head()

,platform,platform_ID
0,Xbox,1
1,PlayStation 2,2
2,PC,3
3,PlayStation 4,4
4,PlayStation 3,5


In [15]:
#### Create Developer table
developers = pd.DataFrame(games.developer.drop_duplicates())
developers = developers.reset_index(drop=True)
ids = pd.Series(range(1, 4080))
developers = pd.concat((developers, ids), axis=1)
developers.columns = ['developer', 'developer_ID']
developers.head()

,developer,developer_ID
0,Criterion Games,1
1,Flight-Plan,2
2,Proper Games,3
3,Pendulo Studios,4
4,damiansommer,5


In [16]:
#### Create Publisher table
publishers = pd.DataFrame(games.publisher.drop_duplicates())
publishers = publishers.reset_index(drop=True)
ids = pd.Series(range(1, 1752))
publishers = pd.concat((publishers, ids), axis=1)
publishers.columns = ['publisher', 'publisher_ID']
publishers.head()

,publisher,publisher_ID
0,EA Games,1
1,Atlus Co.,2
2,Capcom,3
3,Focus Home Interactive,4
4,damiansommer,5


In [17]:
games.shape

(16950, 11)

In [18]:
# Update Game table
games = pd.merge(games, platforms, how = "left", on = ['platform'])
games = pd.merge(games, developers, how = "left", on = ['developer'])
games = pd.merge(games, publishers, how = "left", on = ['publisher'])
games = games.drop(['developer', 'platform', 'publisher'], 1)
games.head()

,game_ID,title,metascore,average_user_score,summary,release_date,ESRB_rating,url,platform_ID,developer_ID,publisher_ID
0,10000,Burnout 3: Takedown,94.0,7.6,Burnout 3 challenges you to crash into (and th...,"Sep 7, 2004",T,https://www.metacritic.com/game/xbox/burnout-3...,1,1,1
1,10001,Eternal Poison,65.0,8.7,Our world descended into chaos the day the dem...,"Nov 11, 2008",T,https://www.metacritic.com/game/playstation-2/...,2,2,2
2,10002,Flock!,65.0,4.6,FLOCK! uses a sophisticated physics engine tha...,"Apr 7, 2009",E,https://www.metacritic.com/game/pc/flock!,3,3,3
3,10003,Yesterday,65.0,6.8,"(Also known as ""New York Crimes"") In New York ...","Mar 22, 2012",NaN,https://www.metacritic.com/game/pc/yesterday,3,4,4
4,10004,The Yawhg,66.0,7.6,The evil Yawhg is returning. How will the town...,"Jun 14, 2013",NaN,https://www.metacritic.com/game/pc/the-yawhg,3,5,5


In [19]:
games = games[['game_ID', 'title', 'platform_ID', 'metascore', 'average_user_score', 'summary',
'developer_ID',  'publisher_ID', 'release_date','ESRB_rating', 'url']]
games.head()

,game_ID,title,platform_ID,metascore,average_user_score,summary,developer_ID,publisher_ID,release_date,ESRB_rating,url
0,10000,Burnout 3: Takedown,1,94.0,7.6,Burnout 3 challenges you to crash into (and th...,1,1,"Sep 7, 2004",T,https://www.metacritic.com/game/xbox/burnout-3...
1,10001,Eternal Poison,2,65.0,8.7,Our world descended into chaos the day the dem...,2,2,"Nov 11, 2008",T,https://www.metacritic.com/game/playstation-2/...
2,10002,Flock!,3,65.0,4.6,FLOCK! uses a sophisticated physics engine tha...,3,3,"Apr 7, 2009",E,https://www.metacritic.com/game/pc/flock!
3,10003,Yesterday,3,65.0,6.8,"(Also known as ""New York Crimes"") In New York ...",4,4,"Mar 22, 2012",NaN,https://www.metacritic.com/game/pc/yesterday
4,10004,The Yawhg,3,66.0,7.6,The evil Yawhg is returning. How will the town...,5,5,"Jun 14, 2013",NaN,https://www.metacritic.com/game/pc/the-yawhg


In [20]:
games.shape

(16950, 11)

## Reviews Table

In [21]:
game_reviews = pd.read_csv('game_reviews.csv')
game_reviews.shape

(1080624, 8)

#### Add Platform_ID Column

In [22]:
game_reviews = pd.merge(platforms, game_reviews, how = "right", on = ['platform'])
game_reviews = game_reviews.drop(['platform'], 1)
game_reviews.head()

,platform_ID,critic_flag,date,review_text,score,title,url,username
0,1,1,NaN,"To say that this game is ""fast"" is an understa...",100.0,Burnout 3: Takedown,https://www.metacritic.com/game/xbox/burnout-3...,GamePro
1,1,1,NaN,Easy controls help you play chicken with oncom...,100.0,Burnout 3: Takedown,https://www.metacritic.com/game/xbox/burnout-3...,Maxim Online
2,1,1,NaN,"With its killer combo of great gameplay, aweso...",100.0,Burnout 3: Takedown,https://www.metacritic.com/game/xbox/burnout-3...,G4 TV
3,1,1,NaN,Every lap is terrifying...Polished to perfecti...,100.0,Burnout 3: Takedown,https://www.metacritic.com/game/xbox/burnout-3...,The Sydney Morning Herald
4,1,1,NaN,The combo of racing and combat is brilliantly ...,100.0,Burnout 3: Takedown,https://www.metacritic.com/game/xbox/burnout-3...,Entertainment Weekly


In [23]:
game_reviews.shape

(1080624, 8)

#### Add Game ID column from Games table:

In [24]:
### Join to get game_ID on game_reviews using the title + platform
game_ids = games[['game_ID', 'title', 'platform_ID']]
game_reviews = pd.merge(game_ids, game_reviews, how = "inner", on = ['title', 'platform_ID'])

#drop extra columns
game_reviews = game_reviews.drop(['title', 'platform_ID', 'url'], 1)
game_reviews.head()

,game_ID,critic_flag,date,review_text,score,username
0,10000,1,NaN,"To say that this game is ""fast"" is an understa...",100.0,GamePro
1,10000,1,NaN,Easy controls help you play chicken with oncom...,100.0,Maxim Online
2,10000,1,NaN,"With its killer combo of great gameplay, aweso...",100.0,G4 TV
3,10000,1,NaN,Every lap is terrifying...Polished to perfecti...,100.0,The Sydney Morning Herald
4,10000,1,NaN,The combo of racing and combat is brilliantly ...,100.0,Entertainment Weekly


In [25]:
game_reviews.shape

(1080624, 6)

#### Handling Missingness in the Reviews table

In [26]:
game_reviews.isnull().sum()
#critic reviews didn't have to have scores, but I decided to still pick up the text
#reviews do not have to have text! And these appear to be from lack of text
#many critic reviews do not have dates, though all user reviews do have dates
#why are usernames missing?

game_ID             0
critic_flag         0
date           193625
review_text        99
score            3170
username            6
dtype: int64

In [27]:
mask = game_reviews[['game_ID', 'username']].isnull().any(axis=1)
game_reviews[['game_ID', 'username']].loc[mask,:]

,game_ID,username
2900,10034,NaN
589522,22206,NaN
592025,22241,NaN
659962,23296,NaN
824112,24756,NaN
838323,24878,NaN


In [28]:
# These are actual usernames, with an account page, not nan values
game_reviews.loc[824112, 'username'] = 'N/A'
game_reviews.loc[2900, 'username'] = 'NULL'
game_reviews.loc[589522, 'username'] = 'NULL'
game_reviews.loc[592025, 'username'] = 'NULL'
game_reviews.loc[659962, 'username'] = 'NULL'
game_reviews.loc[838323, 'username'] = 'NULL'

#### Creating Users table and dropping extra columns on Reviews table:

In [29]:
users = pd.DataFrame(game_reviews[['username', 'critic_flag']].drop_duplicates())
users = users.reset_index(drop=True)
ids = pd.Series(range(1, 323199))
users = pd.concat((ids, users), axis=1)
users.columns = ['user_ID', 'username', 'critic_flag']
users.shape

(323198, 3)

In [30]:
users.head()

,user_ID,username,critic_flag
0,1,GamePro,1
1,2,Maxim Online,1
2,3,G4 TV,1
3,4,The Sydney Morning Herald,1
4,5,Entertainment Weekly,1


In [31]:
# add new user ID to reviews table and droping the username and critic flag
game_reviews = pd.merge(users, game_reviews, how = "right", on = ['username', 'critic_flag'])
game_reviews = game_reviews.drop(['username', 'critic_flag'], 1)
game_reviews.head()

,user_ID,game_ID,date,review_text,score
0,1,10000,NaN,"To say that this game is ""fast"" is an understa...",100.0
1,1,10001,NaN,The gothic-anime art direction and voice actin...,70.0
2,1,10012,NaN,"Series fans'll eat it up , but others may want...",65.0
3,1,10013,NaN,"The result, though a little awkward, is a hybr...",70.0
4,1,10042,NaN,While it is a nice throwback to the retro arca...,10.0


In [32]:
# re-arrange game_review columns for easier reading
game_reviews = game_reviews[['game_ID', 'user_ID', 'score', 'review_text', 'date']]
game_reviews.head()

,game_ID,user_ID,score,review_text,date
0,10000,1,100.0,"To say that this game is ""fast"" is an understa...",NaN
1,10001,1,70.0,The gothic-anime art direction and voice actin...,NaN
2,10012,1,65.0,"Series fans'll eat it up , but others may want...",NaN
3,10013,1,70.0,"The result, though a little awkward, is a hybr...",NaN
4,10042,1,10.0,While it is a nice throwback to the retro arca...,NaN


#### Save New Data Locally

In [35]:
games.to_csv('Processed Data\\games.csv', index = False)
game_reviews.to_csv('Processed Data\\game_reviews.csv', index = False)
genres.to_csv('Processed Data\\genres.csv', index = False)
game_genre_tags.to_csv('Processed Data\\game_genre_tags.csv', index = False)
platforms.to_csv('Processed Data\\platforms.csv', index = False)
developers.to_csv('Processed Data\\developers.csv', index = False)
publishers.to_csv('Processed Data\\publishers.csv', index = False)
users.to_csv('Processed Data\\users.csv', index = False)